### 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 star로 바꿉니다.
ratings.rename(columns={'rating':'star'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.merge(movies, on = 'movie_id')

In [ ]:
idx_to_movie = {i:v for i,v in zip(movies['movie_id'], movies['title'])}
movie_to_idx = {v:i for i,v in zip(movies['movie_id'], movies['title'])}

### 2) 분석
- ratings에 있는 유니크한 영화 개수    
- rating에 있는 유니크한 사용자 수    
- 가장 인기있는 영화 30개(인기순)    

In [5]:
# 유저 수
ratings['user_id'].nunique()

6039

In [6]:
# 영화 수
ratings['movie_id'].nunique()

3628

In [11]:
# movies와 ratings 합치기
new_df=ratings.merge(movies, on = 'movie_id')

In [12]:
# 가장 인기있는 영화 30개
movie_count = new_df.groupby('title')['title'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

'Seven (Se7en) (1995)'   
'Blade Runner (1982)'     
'Matrix, The (1999)'     
'Silence of the Lambs, The (1991)'    
'Godfather, The (1972)'    

In [14]:
my_favorite = ['Seven (Se7en) (1995)' , 'Blade Runner (1982)' ,'Matrix, The (1999)' ,'Silence of the Lambs, The (1991)' ,'Godfather, The (1972)']

my_playlist = pd.DataFrame({'user_id': ['je']*5, 'title':my_favorite,'star':[5]*5})

if not new_df.isin({'user_id':['je']})['user_id'].any():  
    new_df = new_df.append(my_playlist)                           

new_df.tail(10)   

,user_id,movie_id,star,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,je,NaN,5,NaN,Seven (Se7en) (1995),NaN
1,je,NaN,5,NaN,Blade Runner (1982),NaN
2,je,NaN,5,NaN,"Matrix, The (1999)",NaN
3,je,NaN,5,NaN,"Silence of the Lambs, The (1991)",NaN
4,je,NaN,5,NaN,"Godfather, The (1972)",NaN


### 4) CSR matrix를 직접 만들어 봅시다.

In [15]:
df = pd.merge(new_df, movies, on=['title'])

In [16]:
df.tail(10)

,user_id,movie_id_x,star,timestamp,title,genre_x,movie_id_y,genre_y
836473,5494,3530.0,4,9.598163e+08,Smoking/No Smoking (1993),Comedy,3530,Comedy
836474,5556,2198.0,3,9.594455e+08,Modulations (1998),Documentary,2198,Documentary
836475,5949,2198.0,5,9.588464e+08,Modulations (1998),Documentary,2198,Documentary
836476,5675,2703.0,3,9.760291e+08,Broken Vessels (1998),Drama,2703,Drama
836477,5717,2258.0,4,9.585094e+08,Master Ninja I (1984),Action,2258,Action
836478,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western,3607,Comedy|Drama|Western
836479,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror,3026,Horror
836480,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance,690,Romance
836481,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary,2909,Documentary
836482,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama,1360,Drama


In [17]:
df.rename(columns={"movie_id_x":"movie_id", 'genre_x' : 'genre'}, inplace = True)

In [18]:
del df['movie_id_y'], df['genre_y']
df.tail(10)

,user_id,movie_id,star,timestamp,title,genre
836473,5494,3530.0,4,9.598163e+08,Smoking/No Smoking (1993),Comedy
836474,5556,2198.0,3,9.594455e+08,Modulations (1998),Documentary
836475,5949,2198.0,5,9.588464e+08,Modulations (1998),Documentary
836476,5675,2703.0,3,9.760291e+08,Broken Vessels (1998),Drama
836477,5717,2258.0,4,9.585094e+08,Master Ninja I (1984),Action
836478,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836480,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama


In [19]:
user_unique = df['user_id'].unique()
user_to_idx = {v:k for k,v in enumerate(user_unique)}

In [21]:
movie_unique = df['title'].unique()
movie_to_idx = {v:i for i,v in enumerate(movie_unique)}

In [24]:
temp_user_data = df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = df['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df):
    print('movie_id column indexing OK!!')
    df['movie_id2'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

df.head(10)

user_id column indexing Fail!!
movie_id column indexing OK!!


,user_id,movie_id,star,timestamp,title,genre,movie_id2
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
1,1,0,5,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
2,2,0,4,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
3,3,0,4,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
4,4,0,5,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
5,5,0,4,978156168.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
6,6,0,5,982730936.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
7,7,0,5,978136709.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
8,8,0,3,978125194.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
9,9,0,5,978557765.0,One Flew Over the Cuckoo's Nest (1975),Drama,0


In [25]:
df.tail(10)

,user_id,movie_id,star,timestamp,title,genre,movie_id2
836473,1508,3619,4,9.598163e+08,Smoking/No Smoking (1993),Comedy,3619
836474,1532,3620,3,9.594455e+08,Modulations (1998),Documentary,3620
836475,1649,3620,5,9.588464e+08,Modulations (1998),Documentary,3620
836476,1564,3621,3,9.760291e+08,Broken Vessels (1998),Drama,3621
836477,1574,3622,4,9.585094e+08,Master Ninja I (1984),Action,3622
836478,1621,3623,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western,3623
836479,3481,3624,4,9.583469e+08,Slaughterhouse (1987),Horror,3624
836480,3481,3625,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance,3625
836481,4159,3626,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary,3626
836482,1648,3627,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama,3627


In [26]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['movie_id2'].nunique()

csr_data = csr_matrix((df.star, (df.user_id, df.movie_id2)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [27]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# AlternatingLeastSquares 클래스의 init 파라미터

# 1.factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2.regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3.use_gpu : GPU를 사용할 것인지
# 4.iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
# 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [28]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [29]:
csr_data_transpose = csr_data.T
csr_data_transpose
als_model.fit(csr_data_transpose)

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [30]:
je, matrix_m = user_to_idx['je'], movie_to_idx['Blade Runner (1982)']
je_vector, matrix_m_vector = als_model.user_factors[je], als_model.item_factors[matrix_m]

In [31]:
np.dot(je_vector, matrix_m_vector)

0.43794405

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [34]:
favorite_movie = 'Blade Runner (1982)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(680, 0.17590843),
 (1097, 0.09918506),
 (651, 0.09797848),
 (200, 0.09564644),
 (500, 0.08864666),
 (904, 0.086980656),
 (193, 0.08651412),
 (1291, 0.08301666),
 (550, 0.08083045),
 (224, 0.07657381),
 (291, 0.07370612),
 (882, 0.07286978),
 (3156, 0.0719193),
 (1096, 0.07156177),
 (91, 0.07135868)]

In [35]:
# 영화 id를 영화이름으로 매핑

idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]


['Blade Runner (1982)',
 'Brazil (1985)',
 'Aliens (1986)',
 'Terminator, The (1984)',
 'Chinatown (1974)',
 'Clockwork Orange, A (1971)',
 'Alien (1979)',
 'Blood Simple (1984)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'L.A. Confidential (1997)',
 'Maltese Falcon, The (1941)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Running Free (2000)',
 'Manchurian Candidate, The (1962)',
 'Close Encounters of the Third Kind (1977)']

In [36]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [37]:
get_similar_movie('Blade Runner (1982)')

['Blade Runner (1982)',
 'Brazil (1985)',
 'Aliens (1986)',
 'Terminator, The (1984)',
 'Chinatown (1974)',
 'Clockwork Orange, A (1971)',
 'Alien (1979)',
 'Blood Simple (1984)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'L.A. Confidential (1997)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [38]:
user = user_to_idx['je']
# recommend에서는 user*item CSR Matrix를 받습니다.
# filter_already_liked_items : 유저가 이미 평가한 아이템은 제외할지 여부
user_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
user_recommended

[(380, 0.5680125),
 (224, 0.5097517),
 (233, 0.46433693),
 (51, 0.4618761),
 (200, 0.44205987),
 (92, 0.42546105),
 (222, 0.39374352),
 (38, 0.38689667),
 (157, 0.37452927),
 (289, 0.33372182),
 (651, 0.33358908),
 (193, 0.31882864),
 (269, 0.31484824),
 (317, 0.30313033),
 (62, 0.27684072),
 (194, 0.2550809),
 (141, 0.2460501),
 (23, 0.24560502),
 (602, 0.23763594),
 (117, 0.23614189)]

In [39]:
[idx_to_movie[i[0]] for i in user_recommended]

['Godfather: Part II, The (1974)',
 'L.A. Confidential (1997)',
 'Usual Suspects, The (1995)',
 'Fargo (1996)',
 'Terminator, The (1984)',
 'Terminator 2: Judgment Day (1991)',
 'Pulp Fiction (1994)',
 'Sixth Sense, The (1999)',
 'Shawshank Redemption, The (1994)',
 'Reservoir Dogs (1992)',
 'Aliens (1986)',
 'Alien (1979)',
 'GoodFellas (1990)',
 'Twelve Monkeys (1995)',
 'Total Recall (1990)',
 'Die Hard (1988)',
 'Fugitive, The (1993)',
 "Schindler's List (1993)",
 '2001: A Space Odyssey (1968)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)']

In [40]:
Usual_Suspects = movie_to_idx['Usual Suspects, The (1995)']
explain = als_model.explain(user, csr_data, itemid=Usual_Suspects)

In [41]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Seven (Se7en) (1995)', 0.20066676073843664),
 ('Silence of the Lambs, The (1991)', 0.15395954394543826),
 ('Matrix, The (1999)', 0.056872130906936366),
 ('Blade Runner (1982)', 0.034249981828351976),
 ('Godfather, The (1972)', 0.010165097069419586)]

내가 좋아할만한 영화들을 추천받는데,

'Godfather: Part II, The (1974)',
 'L.A. Confidential (1997)',
 'Usual Suspects, The (1995)',
 'Fargo (1996)',
 'Terminator, The (1984)',
 'Terminator 2: Judgment Day (1991)',
 'Pulp Fiction (1994)',
 'Sixth Sense, The (1999)',
 
Godfather2는 후속편이라 추천해줄 수 있는데, Usual Suspects, Terminator, Sixth Sense, Fargo 를 추천해줘서 놀랐다.
세 영화 모두 좋아해서 본 영화들인데 내가 좋아하다고 넣은 영화 5개와 유사하다.
L.A. Confidential, Pulp Fiction 도 유명해서 제목은 들어봤는데, 이걸 추천해주다니.. 나중에 시간나면 봐야겠다.
